# 🚀 Colab GPU Server for NCP-ADS (Flask + ngrok)
This notebook launches a simple **GPU-backed REST API** that Streamlit Cloud can call.
1. Install deps
2. Start Flask on port 5000
3. Expose via ngrok → copy the public URL into Streamlit sidebar


In [ ]:
!pip install -q flask flask-cors pyngrok cupy-cuda12x
try:
    import cudf, cuml  # If RAPIDS wheels are available in your runtime
except Exception:
    print('RAPIDS not preinstalled — proceeding with CuPy-only demo endpoints.')


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import cupy as cp
app = Flask(__name__)
CORS(app)

@app.route('/gpu_task', methods=['POST'])
def gpu_task():
    data = request.json or {}
    arr = cp.array(data.get('values', [1,2,3]))
    result = float(cp.mean(arr))
    return jsonify({'gpu_mean': result})

@app.route('/')
def root():
    return jsonify({'ok': True, 'endpoints': ['/gpu_task']})


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(5000).public_url
print('🌐 Public API endpoint:', public_url)
app.run(host='0.0.0.0', port=5000)
